## Select and analysis dataset

First, we call PreprocessData.select_and_analyze_dataset() to prepare the input dataset and save the train and test data to files.

In [1]:
from PreprocessData import PreprocessData
preprocessData = PreprocessData()
preprocessData.select_and_analyze_dataset()

PreprocessData initialized.
Reading data from ./data/kc_house_data.csv...
Truncating data randomly to 2000 rows
Selecting this columns from the data: ['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'yr_built', 'lat', 'long', 'price']
Removing missing values from columns: ['date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'yr_built', 'lat', 'long', 'price']
Removing outliers values from columns: []
Splitting data: train_data (1600) and test_data (400)
Creating ColumnTransformer
Fit train_data
Transforming train_data and test_data
Train matrix saved to ./data/transformed_train_matrix.csv
Test matrix saved to ./data/transformed_test_matrix.csv
Transformer saved to ./data/transformer.pkl
Executed all subtasks of select and analyze dataset...


## Hyperparameter comparison and selection

We will explore some of the space of hyperparameters, trying different combinations and 
evaluating the quality of the result of the prediction obtained using them.

For that, we load the hyperparameter combinations and the transformed train dataset from files.

In [2]:
import pandas as pd
hyperparameters = pd.read_csv("data/neural_network_parameters.csv")
print(hyperparameters)

   Number of Layers  Layer Structure  Num Epochs  Learning Rate  Momentum  \
0                 4  [22, 64, 32, 1]         150         0.0005      0.95   
1                 2          [22, 1]          80         0.0020      0.85   

  Activation Function  
0                tanh  
1             sigmoid  


In [3]:
X_in, y_in = preprocessData.read_transformed_data_from_file()
print(X_in[:1])
print(y_in[:1])

[[0.30319149 0.375      0.21875    0.10521739 0.00652308 1.
  0.         0.         0.         0.         0.         0.
  1.         0.         0.         0.         0.         0.5
  0.4        0.57391304 0.65979557 0.2078922 ]]
[0.04393443]


For each iteration over the combinations: 
- we create a new instance of the NeuralNet with the hyperparameters,
- call the NeuralNet.fit() function with Y_in (instances) and y_in (ground truth target values) to train our neuronal network,
- call the NeuralNet.predict() function to obtain the estimated target values (y).

In [4]:
from NeuralNet import NeuralNet

neural_net_result_params = {}
for i, params in hyperparameters.iterrows():
    print(f"--- Combination {i} ---")
    neural_net = NeuralNet(
        L = params["Number of Layers"],
        n = eval(params["Layer Structure"]),  # Convert string to list
        n_epochs = params["Num Epochs"],
        learning_rate = params["Learning Rate"],
        momentum = params["Momentum"],
        activation_function = params["Activation Function"],
        validation_split = 0.2
    )

    neural_net.fit(X_in, y_in)
    y_pred = neural_net.predict(X_in)
    epoch_loss = neural_net.loss_epochs()

    neural_net_result_params[i] = {
        "Combination": i,
        "Hyperparameters": params.to_dict(),
        "Y_pred": y_pred,
        "Epoch_loss": epoch_loss
    }

--- Combination 0 ---
NeuralNet initialized with self.L = '4', self.n = '[22, 64, 32, 1]', self.n_epochs = '150', self.learning_rate = '0.0005', self.momentum = '0.95', self.fact = 'tanh', self.validation_split = '0.2'
Executing fit(X, y)
Executing predict(X)
Executing loss_epochs()
--- Combination 1 ---
NeuralNet initialized with self.L = '2', self.n = '[22, 1]', self.n_epochs = '80', self.learning_rate = '0.002', self.momentum = '0.85', self.fact = 'sigmoid', self.validation_split = '0.2'
Executing fit(X, y)
Executing predict(X)
Executing loss_epochs()


neural_net_result_params is a dictionary that contains the info about each combinations of hyperparameter and the result of their predictions. For example we will show the contain of the first element (0):

In [10]:
print(neural_net_result_params[0])

{'Combination': 0, 'Hyperparameters': {'Number of Layers': 4, 'Layer Structure': '[22, 64, 32, 1]', 'Num Epochs': 150, 'Learning Rate': 0.0005, 'Momentum': 0.95, 'Activation Function': 'tanh'}, 'Y_pred': array([[0.05231699],
       [0.07540787],
       [0.12433636],
       ...,
       [0.05068566],
       [0.28153476],
       [0.02944044]]), 'Epoch_loss': (array([0.21527877, 0.1206834 , 0.0643453 , 0.05081552, 0.04446607,
       0.04173823, 0.03972817, 0.03535786, 0.03734698, 0.03331559,
       0.03043819, 0.02790965, 0.02591985, 0.02347408, 0.02205682,
       0.02077442, 0.01924171, 0.02053125, 0.01780949, 0.01626487,
       0.0167151 , 0.01551552, 0.01537815, 0.0130608 , 0.01215309,
       0.01217955, 0.01123702, 0.01075445, 0.01004797, 0.01061427,
       0.00979725, 0.00943311, 0.00972774, 0.00926529, 0.0084988 ,
       0.00852513, 0.00930265, 0.00795946, 0.00763132, 0.00774085,
       0.00782534, 0.00846154, 0.00727647, 0.00794168, 0.00758649,
       0.0070184 , 0.00683956, 0.00773

Now we can calculate MSE(Mean Squared Error), MAE (Mean Absolute Error) and MAPE (Mean Absolute Percentage Error), and compare the results.

After execution the predictions, if  NaN values are generated, we delete the result for that combination

In [11]:
import numpy as np

i = 0
while i < len(neural_net_result_params):
    if np.isnan(neural_net_result_params[i]["Y_pred"]).any():
        print(f"Handling NaN in Combination {neural_net_result_params[i]['Combination']}")
        del neural_net_result_params[i]
    i += 1

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

for key, result in neural_net_result_params.items():
    y_pred = result["Y_pred"]
    
    mse = mean_squared_error(y_in, y_pred)
    mae = mean_absolute_error(y_in, y_pred)
    mape = sum(abs((y_true - y_pred_val) / y_true) for y_true, y_pred_val in zip(y_in, y_pred) if y_true != 0) / len(y_in)

    result["MSE"] = mse
    result["MAE"] = mae
    result["MAPE"] = mape
    

For each combination of hiperparameters we now have values of MSE, MAE and MAPE. 

In [13]:
print(neural_net_result_params[0]["MSE"])

0.004811684208189771


In [ ]:
We can compare the performance of the combinations of hyperparameter:

In [15]:
data = []
for key, result in neural_net_result_params.items():
    hyperparams = result["Hyperparameters"]
    data.append({
        "Combination": key,
        "Number of Layers": hyperparams["Number of Layers"],
        "Layer Structure": hyperparams["Layer Structure"],
        "Num Epochs": hyperparams["Num Epochs"],
        "Learning Rate": hyperparams["Learning Rate"],
        "Momentum": hyperparams["Momentum"],
        "Activation Function": hyperparams["Activation Function"],
        "MAPE": result["MAPE"],
        "MAE": result["MAE"],
        "MSE": result["MSE"]
    })

hyperparameters_performance_results = pd.DataFrame(data)
hyperparameters_performance_results.to_csv("hyperparameters_performance_results.csv", index=False)
print("Data frame saved to 'hyperparameters_performance_results.csv' with the following columns:")
print(hyperparameters_performance_results)


Data frame saved to 'hyperparameters_performance_results.csv' with the following columns:
   Combination  Number of Layers  Layer Structure  Num Epochs  Learning Rate  \
0            0                 4  [22, 64, 32, 1]         150         0.0005   
1            1                 2          [22, 1]          80         0.0020   

   Momentum Activation Function                 MAPE       MAE       MSE  
0      0.95                tanh  [0.617592573727582]  0.028396  0.004812  
1      0.85             sigmoid  [0.846990734960697]  0.041130  0.003942  
